In [77]:
import cv2
import numpy as np
from scipy.spatial import distance
from matplotlib import pyplot as plt
from scipy.interpolate import griddata

In [78]:
def adjust_lightness_to_50(image):
    """Установка яркости L=50 для изображения."""
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
    l, a, b = cv2.split(lab_image)
    l.fill(50)  # Установка яркости L=50
    corrected_lab_image = cv2.merge([l, a, b])
    corrected_rgb_image = cv2.cvtColor(corrected_lab_image, cv2.COLOR_Lab2BGR)
    return corrected_rgb_image

In [79]:
def improve_contrast(image):
    """Улучшение контрастности изображения."""
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
    l, a, b = cv2.split(lab)
    l2 = clahe.apply(l)
    lab2 = cv2.merge([l2, a, b])
    improved_img = cv2.cvtColor(lab2, cv2.COLOR_Lab2BGR)
    return improved_img

In [80]:
def find_nearest_samples_ab(sample_color_lab, test_samples_lab):
    """Поиск ближайших образцов по цвету в пространстве"""
    a, b = sample_color_lab[1], sample_color_lab[2]  # Игнор L
    closest_samples = sorted(test_samples_lab.keys(), key=lambda sample_ab: distance.euclidean((a, b), sample_ab))
    return closest_samples[:2]

In [81]:
# Загрузка и коррекция изображения
image_path = 'image/6.5.png'
image = cv2.imread(image_path)
image_corrected = adjust_lightness_to_50(image)
image_contrasted = improve_contrast(image_corrected)

In [82]:
gray = cv2.cvtColor(image_corrected, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (9, 9), 0)
circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=100, param1=50, param2=30, minRadius=20, maxRadius=200)

In [83]:
mean_color_lab = None
if circles is not None:
    circles = np.uint16(np.around(circles))
    for circle in circles[0, :]:
        x, y, r = circle
        mask = np.zeros(gray.shape, dtype="uint8")
        cv2.circle(mask, (x, y), r // 2, 255, -1)
        mean_val = cv2.mean(image_corrected, mask=mask)
        lab_image = cv2.cvtColor(np.uint8([[mean_val[:3]]]), cv2.COLOR_BGR2Lab)[0][0]
        print("Средний LAB цвет центральной части круга:", lab_image)
        mean_color_lab = lab_image
else:
    print("Круги не обнаружены.")

Круги не обнаружены.


In [84]:
# Словарь с данными
test_samples_lab = {
    (235, 223): 8.0,
    (232, 217): 7.7,
    (222, 221): 7.5,
    (222, 226): 7.2,
    (228, 234): 6.9,
    (236, 232): 6.5,
    (220, 235): 6.3,
    (207, 234): 6.0,
    (174, 234): 5.7,
}


In [85]:
if mean_color_lab is not None:
    points = np.array(list(test_samples_lab.keys()))
    values = np.array(list(test_samples_lab.values()))
    estimated_ph = griddata(points, values, [mean_color_lab], method='nearest')[0]
    print("Оценочное значение pH:", estimated_ph)
else:
    print("Невозможно оценить pH: нет среднего цвета.")

Невозможно оценить pH: нет среднего цвета.
